In [1]:
import numpy as np
import json
import matplotlib.pyplot as plt
import os
import cv2
import datetime
import dateutil.relativedelta
import math

In [2]:
ts = datetime.datetime.fromtimestamp(1619195309.272862)


In [5]:
def draw_xywha(im, num, x, y, w, h, angle, color=(0,255,0), linewidth=5):
    '''
    im: image numpy array, shape(h,w,3), RGB
    angle: degree
    '''
    font = cv2.FONT_HERSHEY_SIMPLEX
    c, s = np.cos(angle/180*np.pi), np.sin(angle/180*np.pi)
    R = np.asarray([[c, s], [-s, c]])
    pts = np.asarray([[-w, -h], [w, -h], [w, h], [-w, h]])
    rot_pts = []
    for pt in pts:
        rot_pts.append(([2*x, 2*y] + pt @ R).astype(int))
    contours = np.array([rot_pts[0], rot_pts[1], rot_pts[2], rot_pts[3]])
    cv2.polylines(im, [contours], isClosed=True, color=color,
                thickness=linewidth, lineType=cv2.LINE_4)
    cv2.putText(im,str(num),(int(2 *x)-15, int(2 *y)), font, 2, (0, 255, 0), 3, cv2.LINE_AA)

def drawing_bb(folder_path, video_name, RP1, RP2):

    out = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v') ,5, (1024, 1024))
    allFiles = os.listdir(folder_path)
    jsonList = []
    files = []
    RP = RP1 + RP2
    RP = sorted(RP)
    RP_index = 0
    RP = [f for f in RP if '._' not in f]
    print(RP)
    for f in allFiles:
        if ".jpg" in f:
            if "._" not in f:
                files.append(f)
        if '.json' in f:
            if '._' not in f:
                jsonList.append(f)
#     print(files)
    jsonList = sorted(jsonList)
    files = sorted(files)
#     print(len(jsonList), len(files))
    RP1count = 0
    RP2count = 0
    for i in range(len(files)):
        loc =  folder_path + files[i]
        jloc = folder_path + jsonList[i]
        with open(jloc) as f:
            cam1 = json.load(f)
        font = cv2.FONT_HERSHEY_SIMPLEX
        image = cv2.imread(loc)
        boxes = list(cam1["Bounding Box"].keys())
        count = 0
        for b in boxes:
            count+=1
            draw_xywha(image, count, cam1["Bounding Box"][b][0], cam1["Bounding Box"][b][1], cam1["Bounding Box"][b][2], cam1["Bounding Box"][b][3], cam1["Bounding Box"][b][4], color=(0,255,0), linewidth=5)
#             cv2.putText(image,str(count),(int(cam1["Bounding Box"][b][0])-15, int(cam1["Bounding Box"][b][1])), font, 2, (255, 0, 0), 3, cv2.LINE_AA)
        cv2.putText(image,"People#: " + str(cam1["Num People"]),(1600, 50), font, 2, (0, 0, 255), 3, cv2.LINE_AA)
        ts = datetime.datetime.fromtimestamp(1619698543.2022455)
        ts2 = datetime.datetime.fromtimestamp(float(cam1['Timestamp']))
        diff = dateutil.relativedelta.relativedelta(ts2, ts)
        print(math.floor((diff.hours*3600 + diff.minutes * 60 + diff.seconds)/60), (diff.hours*3600 + diff.minutes * 60 + diff.seconds)%60)    
        cv2.putText(image,"since: " + str(math.floor((diff.hours*3600 + diff.minutes * 60 + diff.seconds)/60))+"min "+str(math.floor((diff.hours*3600 + diff.minutes * 60 + diff.seconds)%60))+"sec",(1600, 120), font, 1.3, (0, 0, 255), 3, cv2.LINE_AA)
        
        RP_relative = float(RP[RP_index].replace('.json', ''))
        if float(cam1['Timestamp']) > RP_relative:
            if RP[RP_index] in RP1:
                with open('/home/team19/Fusion/RP1/' + RP[RP_index]) as f:
                    RPinfo = json.load(f)
                RP1count = int(RPinfo['count'])
            elif RP[RP_index] in RP2:
                with open('/home/team19/Fusion/RP2/' + RP[RP_index]) as f:
                    RPinfo = json.load(f)
                RP2count = int(RPinfo['count'])
            if RP_index < len(RP)-1:
                RP_index += 1
        print("TDS1: " + str(RP1count), "TDS2: " + str(RP2count))
        cv2.putText(image, "TDS1: " + str(RP2count),(1700, 1900), font, 1.3, (0, 0, 255), 3, cv2.LINE_AA)
        cv2.putText(image, "TDS2: " + str(RP1count),(1700, 1950), font, 1.3, (0, 0, 255), 3, cv2.LINE_AA)
        resized = cv2.resize(image, (1024,1024))
        out.write(resized)
        
#     for data_file in sorted(files):
#         loc = folder_path+data_file
#         print(loc)
#         image = cv2.imread(loc)
        
#         resized = cv2.resize(image, (1024,1024))

#         out.write(resized)

        
    out.release()

In [6]:
folder_path = '/home/team19/Desktop/Axis_DL/Detection/YOLO/04-29-2021-08:15:41/Camera 1/'
video_name = 'trial.mp4'
RP1 = os.listdir('/home/team19/Fusion/RP1/')
RP2 = os.listdir('/home/team19/Fusion/RP2/')
drawing_bb(folder_path, video_name, RP1, RP2)

['1619700904.061.json', '1619700905.463.json', '1619702023.417.json', '1619704060.201.json', '1619704416.65.json', '1619704433.259.json', '1619704636.96.json', '1619707633.657.json', '1619708259.245.json', '1619708266.408.json', '1619709667.358.json']
0 0
TDS1: 0 TDS2: 0
0 1
TDS1: 0 TDS2: 0
0 3
TDS1: 0 TDS2: 0
0 5
TDS1: 0 TDS2: 0
0 7
TDS1: 0 TDS2: 0
0 9
TDS1: 0 TDS2: 0
0 10
TDS1: 0 TDS2: 0
0 12
TDS1: 0 TDS2: 0
0 14
TDS1: 0 TDS2: 0
0 16
TDS1: 0 TDS2: 0
0 18
TDS1: 0 TDS2: 0
0 20
TDS1: 0 TDS2: 0
0 21
TDS1: 0 TDS2: 0
0 23
TDS1: 0 TDS2: 0
0 25
TDS1: 0 TDS2: 0
0 27
TDS1: 0 TDS2: 0
0 29
TDS1: 0 TDS2: 0
0 30
TDS1: 0 TDS2: 0
0 32
TDS1: 0 TDS2: 0
0 34
TDS1: 0 TDS2: 0
0 36
TDS1: 0 TDS2: 0
0 38
TDS1: 0 TDS2: 0
0 40
TDS1: 0 TDS2: 0
0 41
TDS1: 0 TDS2: 0
0 43
TDS1: 0 TDS2: 0
0 45
TDS1: 0 TDS2: 0
0 47
TDS1: 0 TDS2: 0
0 49
TDS1: 0 TDS2: 0
0 51
TDS1: 0 TDS2: 0
0 53
TDS1: 0 TDS2: 0
0 54
TDS1: 0 TDS2: 0
0 56
TDS1: 0 TDS2: 0
0 58
TDS1: 0 TDS2: 0
1 0
TDS1: 0 TDS2: 0
1 2
TDS1: 0 TDS2: 0
1 4
TDS1: 0 TDS2: 0
1

11 45
TDS1: 0 TDS2: 0
11 47
TDS1: 0 TDS2: 0
11 48
TDS1: 0 TDS2: 0
11 50
TDS1: 0 TDS2: 0
11 52
TDS1: 0 TDS2: 0
11 54
TDS1: 0 TDS2: 0
11 56
TDS1: 0 TDS2: 0
11 58
TDS1: 0 TDS2: 0
12 0
TDS1: 0 TDS2: 0
12 1
TDS1: 0 TDS2: 0
12 3
TDS1: 0 TDS2: 0
12 5
TDS1: 0 TDS2: 0
12 7
TDS1: 0 TDS2: 0
12 9
TDS1: 0 TDS2: 0
12 11
TDS1: 0 TDS2: 0
12 13
TDS1: 0 TDS2: 0
12 15
TDS1: 0 TDS2: 0
12 16
TDS1: 0 TDS2: 0
12 18
TDS1: 0 TDS2: 0
12 20
TDS1: 0 TDS2: 0
12 22
TDS1: 0 TDS2: 0
12 24
TDS1: 0 TDS2: 0
12 26
TDS1: 0 TDS2: 0
12 28
TDS1: 0 TDS2: 0
12 29
TDS1: 0 TDS2: 0
12 31
TDS1: 0 TDS2: 0
12 33
TDS1: 0 TDS2: 0
12 35
TDS1: 0 TDS2: 0
12 37
TDS1: 0 TDS2: 0
12 39
TDS1: 0 TDS2: 0
12 41
TDS1: 0 TDS2: 0
12 43
TDS1: 0 TDS2: 0
12 44
TDS1: 0 TDS2: 0
12 46
TDS1: 0 TDS2: 0
12 48
TDS1: 0 TDS2: 0
12 50
TDS1: 0 TDS2: 0
12 52
TDS1: 0 TDS2: 0
12 54
TDS1: 0 TDS2: 0
12 56
TDS1: 0 TDS2: 0
12 57
TDS1: 0 TDS2: 0
12 59
TDS1: 0 TDS2: 0
13 1
TDS1: 0 TDS2: 0
13 3
TDS1: 0 TDS2: 0
13 5
TDS1: 0 TDS2: 0
13 7
TDS1: 0 TDS2: 0
13 9
TDS1: 0 TDS2: 0

23 34
TDS1: 0 TDS2: 0
23 36
TDS1: 0 TDS2: 0
23 38
TDS1: 0 TDS2: 0
23 40
TDS1: 0 TDS2: 0
23 41
TDS1: 0 TDS2: 0
23 43
TDS1: 0 TDS2: 0
23 45
TDS1: 0 TDS2: 0
23 47
TDS1: 0 TDS2: 0
23 49
TDS1: 0 TDS2: 0
23 51
TDS1: 0 TDS2: 0
23 53
TDS1: 0 TDS2: 0
23 55
TDS1: 0 TDS2: 0
23 56
TDS1: 0 TDS2: 0
23 58
TDS1: 0 TDS2: 0
24 0
TDS1: 0 TDS2: 0
24 2
TDS1: 0 TDS2: 0
24 4
TDS1: 0 TDS2: 0
24 6
TDS1: 0 TDS2: 0
24 8
TDS1: 0 TDS2: 0
24 10
TDS1: 0 TDS2: 0
24 11
TDS1: 0 TDS2: 0
24 13
TDS1: 0 TDS2: 0
24 15
TDS1: 0 TDS2: 0
24 17
TDS1: 0 TDS2: 0
24 19
TDS1: 0 TDS2: 0
24 21
TDS1: 0 TDS2: 0
24 23
TDS1: 0 TDS2: 0
24 25
TDS1: 0 TDS2: 0
24 26
TDS1: 0 TDS2: 0
24 28
TDS1: 0 TDS2: 0
24 30
TDS1: 0 TDS2: 0
24 32
TDS1: 0 TDS2: 0
24 34
TDS1: 0 TDS2: 0
24 36
TDS1: 0 TDS2: 0
24 38
TDS1: 0 TDS2: 0
24 40
TDS1: 0 TDS2: 0
24 41
TDS1: 0 TDS2: 0
24 43
TDS1: 0 TDS2: 0
24 45
TDS1: 0 TDS2: 0
24 47
TDS1: 0 TDS2: 0
24 49
TDS1: 0 TDS2: 0
24 51
TDS1: 0 TDS2: 0
24 53
TDS1: 0 TDS2: 0
24 55
TDS1: 0 TDS2: 0
24 56
TDS1: 0 TDS2: 0
24 58
TDS1: 0 T

35 23
TDS1: 0 TDS2: 0
35 25
TDS1: 0 TDS2: 0
35 26
TDS1: 0 TDS2: 0
35 28
TDS1: 0 TDS2: 0
35 30
TDS1: 0 TDS2: 0
35 32
TDS1: 0 TDS2: 0
35 34
TDS1: 0 TDS2: 0
35 36
TDS1: 0 TDS2: 0
35 38
TDS1: 0 TDS2: 0
35 39
TDS1: 0 TDS2: 0
35 41
TDS1: 0 TDS2: 0
35 43
TDS1: 0 TDS2: 0
35 45
TDS1: 0 TDS2: 0
35 47
TDS1: 0 TDS2: 0
35 49
TDS1: 0 TDS2: 0
35 51
TDS1: 0 TDS2: 0
35 52
TDS1: 0 TDS2: 0
35 54
TDS1: 0 TDS2: 0
35 56
TDS1: 0 TDS2: 0
35 58
TDS1: 0 TDS2: 0
36 0
TDS1: 0 TDS2: 0
36 2
TDS1: 0 TDS2: 0
36 3
TDS1: 0 TDS2: 0
36 5
TDS1: 0 TDS2: 0
36 7
TDS1: 0 TDS2: 0
36 9
TDS1: 0 TDS2: 0
36 11
TDS1: 0 TDS2: 0
36 13
TDS1: 0 TDS2: 0
36 15
TDS1: 0 TDS2: 0
36 16
TDS1: 0 TDS2: 0
36 18
TDS1: 0 TDS2: 0
36 20
TDS1: 0 TDS2: 0
36 22
TDS1: 0 TDS2: 0
36 24
TDS1: 0 TDS2: 0
36 26
TDS1: 0 TDS2: 0
36 28
TDS1: 0 TDS2: 0
36 29
TDS1: 0 TDS2: 0
36 31
TDS1: 0 TDS2: 0
36 33
TDS1: 0 TDS2: 0
36 35
TDS1: 0 TDS2: 0
36 37
TDS1: 0 TDS2: 0
36 39
TDS1: 0 TDS2: 0
36 41
TDS1: 0 TDS2: 0
36 42
TDS1: 0 TDS2: 0
36 44
TDS1: 0 TDS2: 0
36 46
TDS1: 0 TD

47 11
TDS1: 2 TDS2: 0
47 13
TDS1: 2 TDS2: 0
47 15
TDS1: 2 TDS2: 0
47 17
TDS1: 2 TDS2: 0
47 19
TDS1: 2 TDS2: 0
47 21
TDS1: 2 TDS2: 0
47 23
TDS1: 2 TDS2: 0
47 25
TDS1: 2 TDS2: 0
47 26
TDS1: 2 TDS2: 0
47 28
TDS1: 2 TDS2: 0
47 30
TDS1: 2 TDS2: 0
47 32
TDS1: 2 TDS2: 0
47 34
TDS1: 2 TDS2: 0
47 36
TDS1: 2 TDS2: 0
47 38
TDS1: 2 TDS2: 0
47 40
TDS1: 2 TDS2: 0
47 41
TDS1: 2 TDS2: 0
47 43
TDS1: 2 TDS2: 0
47 45
TDS1: 2 TDS2: 0
47 47
TDS1: 2 TDS2: 0
47 49
TDS1: 2 TDS2: 0
47 51
TDS1: 2 TDS2: 0
47 53
TDS1: 2 TDS2: 0
47 55
TDS1: 2 TDS2: 0
47 57
TDS1: 2 TDS2: 0
47 58
TDS1: 2 TDS2: 0
48 0
TDS1: 2 TDS2: 0
48 2
TDS1: 2 TDS2: 0
48 4
TDS1: 2 TDS2: 0
48 6
TDS1: 2 TDS2: 0
48 8
TDS1: 2 TDS2: 0
48 10
TDS1: 2 TDS2: 0
48 12
TDS1: 2 TDS2: 0
48 13
TDS1: 2 TDS2: 0
48 15
TDS1: 2 TDS2: 0
48 17
TDS1: 2 TDS2: 0
48 19
TDS1: 2 TDS2: 0
48 21
TDS1: 2 TDS2: 0
48 23
TDS1: 2 TDS2: 0
48 25
TDS1: 2 TDS2: 0
48 27
TDS1: 2 TDS2: 0
48 28
TDS1: 2 TDS2: 0
48 30
TDS1: 2 TDS2: 0
48 32
TDS1: 2 TDS2: 0
48 34
TDS1: 2 TDS2: 0
48 36
TDS1: 2 T

...

170 23
TDS1: 4 TDS2: -2
170 25
TDS1: 4 TDS2: -2
170 27
TDS1: 4 TDS2: -2
170 29
TDS1: 4 TDS2: -2
170 31
TDS1: 4 TDS2: -2
170 33
TDS1: 4 TDS2: -2
170 35
TDS1: 4 TDS2: -2
170 37
TDS1: 4 TDS2: -2
170 38
TDS1: 4 TDS2: -2
170 40
TDS1: 4 TDS2: -2
170 42
TDS1: 4 TDS2: -2
170 44
TDS1: 4 TDS2: -2
170 46
TDS1: 4 TDS2: -2
170 48
TDS1: 4 TDS2: -2
170 49
TDS1: 4 TDS2: -2
170 51
TDS1: 4 TDS2: -2
170 53
TDS1: 4 TDS2: -2
170 55
TDS1: 4 TDS2: -2
170 57
TDS1: 4 TDS2: -2
170 59
TDS1: 4 TDS2: -2
171 1
TDS1: 4 TDS2: -2
171 2
TDS1: 4 TDS2: -2
171 4
TDS1: 4 TDS2: -2
171 6
TDS1: 4 TDS2: -2
171 8
TDS1: 4 TDS2: -2
171 10
TDS1: 4 TDS2: -2
171 12
TDS1: 4 TDS2: -2
171 14
TDS1: 4 TDS2: -2
171 15
TDS1: 4 TDS2: -2
171 17
TDS1: 4 TDS2: -2
171 19
TDS1: 4 TDS2: -2
171 21
TDS1: 4 TDS2: -2
171 23
TDS1: 4 TDS2: -2
171 25
TDS1: 4 TDS2: -2
171 27
TDS1: 4 TDS2: -2
171 29
TDS1: 4 TDS2: -2
171 31
TDS1: 4 TDS2: -2
171 32
TDS1: 4 TDS2: -2
171 34
TDS1: 4 TDS2: -2
171 36
TDS1: 4 TDS2: -2
171 38
TDS1: 4 TDS2: -2
171 40
TDS1: 4 TDS2: 

181 6
TDS1: 4 TDS2: -2
181 8
TDS1: 4 TDS2: -2
181 10
TDS1: 4 TDS2: -2
181 12
TDS1: 4 TDS2: -2
181 14
TDS1: 4 TDS2: -2
181 16
TDS1: 4 TDS2: -2
181 17
TDS1: 4 TDS2: -2
181 19
TDS1: 4 TDS2: -2
181 21
TDS1: 4 TDS2: -2
181 23
TDS1: 4 TDS2: -2
181 25
TDS1: 4 TDS2: -2
181 27
TDS1: 4 TDS2: -2
181 29
TDS1: 4 TDS2: -2
181 30
TDS1: 4 TDS2: -2
181 32
TDS1: 4 TDS2: -2
181 34
TDS1: 4 TDS2: -2
181 36
TDS1: 4 TDS2: -2
181 38
TDS1: 4 TDS2: -2
181 40
TDS1: 4 TDS2: -2
181 42
TDS1: 4 TDS2: -2
181 44
TDS1: 4 TDS2: -2
181 45
TDS1: 4 TDS2: -2
181 47
TDS1: 4 TDS2: -2
181 49
TDS1: 4 TDS2: -2
181 51
TDS1: 4 TDS2: -2
181 53
TDS1: 4 TDS2: -2
181 55
TDS1: 4 TDS2: -2
181 57
TDS1: 4 TDS2: -2
181 58
TDS1: 4 TDS2: -2
182 0
TDS1: 4 TDS2: -2
182 2
TDS1: 4 TDS2: -2
182 4
TDS1: 4 TDS2: -2
182 6
TDS1: 4 TDS2: -2
182 8
TDS1: 4 TDS2: -2
182 10
TDS1: 4 TDS2: -2
182 11
TDS1: 4 TDS2: -2
182 13
TDS1: 4 TDS2: -2
182 15
TDS1: 4 TDS2: -2
182 17
TDS1: 4 TDS2: -2
182 19
TDS1: 4 TDS2: -2
182 21
TDS1: 4 TDS2: -2
182 23
TDS1: 4 TDS2: -2

191 50
TDS1: 5 TDS2: -2
191 52
TDS1: 5 TDS2: -2
191 54
TDS1: 5 TDS2: -2
191 55
TDS1: 5 TDS2: -2
191 57
TDS1: 5 TDS2: -2
191 59
TDS1: 5 TDS2: -2
192 1
TDS1: 5 TDS2: -2
192 3
TDS1: 5 TDS2: -2
192 5
TDS1: 5 TDS2: -2
192 7
TDS1: 5 TDS2: -2
192 8
TDS1: 5 TDS2: -2
192 10
TDS1: 5 TDS2: -2
192 12
TDS1: 5 TDS2: -2
192 14
TDS1: 5 TDS2: -2
192 16
TDS1: 5 TDS2: -2
192 18
TDS1: 5 TDS2: -2
192 20
TDS1: 5 TDS2: -2
192 21
TDS1: 5 TDS2: -2
192 23
TDS1: 5 TDS2: -2
192 25
TDS1: 5 TDS2: -2
192 27
TDS1: 5 TDS2: -2
192 29
TDS1: 5 TDS2: -2
192 31
TDS1: 5 TDS2: -2
192 33
TDS1: 5 TDS2: -2
192 35
TDS1: 5 TDS2: -2
192 37
TDS1: 5 TDS2: -2
192 38
TDS1: 5 TDS2: -2
192 40
TDS1: 5 TDS2: -2
192 42
TDS1: 5 TDS2: -2
192 44
TDS1: 5 TDS2: -2
192 46
TDS1: 5 TDS2: -2
192 48
TDS1: 5 TDS2: -2
192 50
TDS1: 5 TDS2: -2
192 52
TDS1: 5 TDS2: -2
192 53
TDS1: 5 TDS2: -2
192 55
TDS1: 5 TDS2: -2
192 57
TDS1: 5 TDS2: -2
192 59
TDS1: 5 TDS2: -2
193 1
TDS1: 5 TDS2: -2
193 3
TDS1: 5 TDS2: -2
193 5
TDS1: 5 TDS2: -2
193 7
TDS1: 5 TDS2: -2
1

202 34
TDS1: 5 TDS2: -2
202 35
TDS1: 5 TDS2: -2
202 37
TDS1: 5 TDS2: -2
202 39
TDS1: 5 TDS2: -2
202 41
TDS1: 5 TDS2: -2
202 43
TDS1: 5 TDS2: -2
202 45
TDS1: 5 TDS2: -2
202 47
TDS1: 5 TDS2: -2
202 49
TDS1: 5 TDS2: -2
202 50
TDS1: 5 TDS2: -2
202 52
TDS1: 5 TDS2: -2
202 54
TDS1: 5 TDS2: -2
202 56
TDS1: 5 TDS2: -2
202 58
TDS1: 5 TDS2: -2
203 0
TDS1: 5 TDS2: -2
203 2
TDS1: 5 TDS2: -2
203 3
TDS1: 5 TDS2: -2
203 5
TDS1: 5 TDS2: -2
203 7
TDS1: 5 TDS2: -2
203 9
TDS1: 5 TDS2: -2
203 11
TDS1: 5 TDS2: -2
203 13
TDS1: 5 TDS2: -2
203 15
TDS1: 5 TDS2: -2
203 17
TDS1: 5 TDS2: -2
203 18
TDS1: 5 TDS2: -2
203 20
TDS1: 5 TDS2: -2
203 22
TDS1: 5 TDS2: -2
203 24
TDS1: 5 TDS2: -2
203 26
TDS1: 5 TDS2: -2
203 28
TDS1: 5 TDS2: -2
203 30
TDS1: 5 TDS2: -2
203 31
TDS1: 5 TDS2: -2
203 33
TDS1: 5 TDS2: -2
203 35
TDS1: 5 TDS2: -2
203 37
TDS1: 5 TDS2: -2
203 39
TDS1: 5 TDS2: -2
203 41
TDS1: 5 TDS2: -2
203 43
TDS1: 5 TDS2: -2
203 44
TDS1: 5 TDS2: -2
203 46
TDS1: 5 TDS2: -2
203 48
TDS1: 5 TDS2: -2
203 50
TDS1: 5 TDS2: -

213 19
TDS1: 5 TDS2: -2
213 21
TDS1: 5 TDS2: -2
213 22
TDS1: 5 TDS2: -2
213 24
TDS1: 5 TDS2: -2
213 26
TDS1: 5 TDS2: -2
213 28
TDS1: 5 TDS2: -2
213 30
TDS1: 5 TDS2: -2
213 32
TDS1: 5 TDS2: -2
213 34
TDS1: 5 TDS2: -2
213 36
TDS1: 5 TDS2: -2
213 37
TDS1: 5 TDS2: -2
213 39
TDS1: 5 TDS2: -2
213 41
TDS1: 5 TDS2: -2
213 43
TDS1: 5 TDS2: -2
213 45
TDS1: 5 TDS2: -2
213 47
TDS1: 5 TDS2: -2
213 49
TDS1: 5 TDS2: -2
213 51
TDS1: 5 TDS2: -2
213 53
TDS1: 5 TDS2: -2
213 55
TDS1: 5 TDS2: -2
213 56
TDS1: 5 TDS2: -2
213 58
TDS1: 5 TDS2: -2
214 0
TDS1: 5 TDS2: -2
214 2
TDS1: 5 TDS2: -2
214 4
TDS1: 5 TDS2: -2
214 6
TDS1: 5 TDS2: -2
214 8
TDS1: 5 TDS2: -2
214 10
TDS1: 5 TDS2: -2
214 11
TDS1: 5 TDS2: -2
214 13
TDS1: 5 TDS2: -2
214 15
TDS1: 5 TDS2: -2
214 17
TDS1: 5 TDS2: -2
214 19
TDS1: 5 TDS2: -2
214 21
TDS1: 5 TDS2: -2
214 23
TDS1: 5 TDS2: -2
214 25
TDS1: 5 TDS2: -2
214 26
TDS1: 5 TDS2: -2
214 28
TDS1: 5 TDS2: -2
214 30
TDS1: 5 TDS2: -2
214 32
TDS1: 5 TDS2: -2
214 34
TDS1: 5 TDS2: -2
214 36
TDS1: 5 TDS2: 